In [1]:
import os 
from dotenv import load_dotenv
load_dotenv('../../.env')

model=os.environ.get('model')
print(model)

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get('AZURE_API_BASE')
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get('AZURE_API_KEY')
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ.get('AZURE_API_VERSION')

azure/gpt-4


In [2]:
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from typing import List, Dict, Any

In [3]:
class TravelCrew:
    def __init__(self):
        # Create specialized travel agents
        self.flight_agent = Agent(
            role='Flight Expert',
            goal='Help users with flight-related queries and bookings',
            backstory=dedent("""
                You are an expert in flight bookings and airline operations. 
                You have extensive knowledge of airlines, routes, and booking processes.
            """),
            verbose=True,
            llm=model,
        )

        self.train_agent = Agent(
            role='Train Expert',
            goal='Help users with train-related queries and bookings',
            backstory=dedent("""
                You are an expert in train travel and railway systems.
                You have deep knowledge of train routes, schedules, and booking procedures.
            """),
            verbose=True,
            llm=model,
        )

        self.travel_assistant = Agent(
            role='General Travel Assistant',
            goal='Help with general travel planning and queries',
            backstory=dedent("""
                You are a knowledgeable travel assistant with expertise in 
                various aspects of travel planning including accommodations,
                local transportation, and tourist activities.
            """),
            verbose=True,
            llm=model,
        )

In [4]:
class FinanceCrew:
    def __init__(self):
        # Create specialized finance agents
        self.stock_agent = Agent(
            role='Stock Market Expert',
            goal='Provide stock market analysis and investment advice',
            backstory=dedent("""
                You are a seasoned stock market analyst with extensive experience
                in equity markets and investment strategies.
            """),
            verbose=True,
            llm=model,
        )

        self.mutual_fund_agent = Agent(
            role='Mutual Fund Expert',
            goal='Provide mutual fund investment advice',
            backstory=dedent("""
                You are an expert in mutual funds with deep knowledge of 
                different fund types, risk assessment, and portfolio management.
            """),
            verbose=True,
            llm=model,
        )

        self.finance_assistant = Agent(
            role='General Finance Assistant',
            goal='Help with general financial queries',
            backstory=dedent("""
                You are a knowledgeable financial advisor who can help with
                basic financial planning and general money management questions.
            """),
            verbose=True,
            llm=model,
        )

In [5]:
class SupervisorSystem:
    def __init__(self):
        # Initialize specialized crews
        self.travel_crew = TravelCrew()
        self.finance_crew = FinanceCrew()

        # Create supervisor agents
        self.travel_supervisor = Agent(
            role='Travel Supervisor',
            goal='Route and manage travel-related queries',
            backstory=dedent("""
                You are a supervisor responsible for analyzing travel-related queries
                and directing them to the appropriate specialized agent.
            """),
            verbose=True,
            llm=model,
        )

        self.finance_supervisor = Agent(
            role='Finance Supervisor',
            goal='Route and manage finance-related queries',
            backstory=dedent("""
                You are a supervisor responsible for analyzing finance-related queries
                and directing them to the appropriate specialized agent.
            """),
            verbose=True,
            llm=model,
        )

        self.main_supervisor = Agent(
            role='Main Supervisor',
            goal='Route queries to appropriate department',
            backstory=dedent("""
                You are the main supervisor responsible for analyzing incoming queries
                and routing them to either the travel or finance department.
            """),
            verbose=True,
            llm=model,
        )

    def _create_travel_tasks(self, query: str) -> List[Task]:
        # Analyze query to determine specific travel agent
        if "flight" in query.lower() or "airline" in query.lower():
            agent = self.travel_crew.flight_agent
        elif "train" in query.lower() or "railway" in query.lower():
            agent = self.travel_crew.train_agent
        else:
            agent = self.travel_crew.travel_assistant

        return [
            Task(
                description=f"Process travel query: {query}",
                expected_output="A satisfactory resolution of the query",
                agent=agent
            )
        ]

    def _create_finance_tasks(self, query: str) -> List[Task]:
        # Analyze query to determine specific finance agent
        if "stock" in query.lower() or "shares" in query.lower():
            agent = self.finance_crew.stock_agent
        elif "mutual fund" in query.lower() or "mf" in query.lower():
            agent = self.finance_crew.mutual_fund_agent
        else:
            agent = self.finance_crew.finance_assistant

        return [
            Task(
                description=f"Process finance query: {query}",
                expected_output="A satisfactory resolution of the query",
                agent=agent
            )
        ]

    def process_query(self, query: str) -> str:
        # First, determine if query is travel or finance related
        travel_keywords = {"travel", "flight", "train", "hotel", "booking", "trip", "vacation"}
        finance_keywords = {"money", "invest", "stock", "fund", "financial", "market"}
        
        query_lower = query.lower()
        is_travel = any(keyword in query_lower for keyword in travel_keywords)
        is_finance = any(keyword in query_lower for keyword in finance_keywords)

        # Create appropriate tasks based on query type
        if is_travel:
            tasks = self._create_travel_tasks(query)
            supervisor = self.travel_supervisor
        elif is_finance:
            tasks = self._create_finance_tasks(query)
            supervisor = self.finance_supervisor
        else:
            # Default to travel for unclassified queries
            tasks = self._create_travel_tasks(query)
            supervisor = self.travel_supervisor

        # Create crew with appropriate supervisor and tasks
        crew = Crew(
            agents=[supervisor],
            tasks=tasks,
            process=Process.sequential
        )

        # Execute the tasks
        result = crew.kickoff()
        return result

In [6]:
def main():
    # Initialize the system
    system = SupervisorSystem()

    # Example queries
    example_queries = [
        "I need to book a flight from New York to London",
        "What are the best mutual funds to invest in?",
        "Help me plan a vacation in Europe",
        "Should I invest in tech stocks?",
        "I need a train ticket to Boston"
    ]

    # Process each query
    for query in example_queries:
        print(f"\nProcessing query: {query}")
        result = system.process_query(query)
        print(f"Response: {result}")

In [7]:
if __name__ == "__main__":
    main()


Processing query: I need to book a flight from New York to London
# Agent: Flight Expert
## Task: Process travel query: I need to book a flight from New York to London


Overriding of current TracerProvider is not allowed




# Agent: Flight Expert
## Final Answer: 
To book a flight from New York to London, I would need to gather more specific information from you to provide the most complete and satisfactory answer possible. Here are the details I would need:

1. Your preferred dates of travel for both departure and return, if it's a round trip.
2. The number of passengers that will be flying.
3. Your preferred airline(s), if any, or if you're looking for the most cost-effective option.
4. Your preferred class of service - economy, business, or first class.
5. Any specific requirements such as direct flights only, flight timing preferences, or additional services.
   
Once I have this information, I would be able to look into different airlines that operate between New York and London, such as British Airways, Virgin Atlantic, American Airlines, and Delta Air Lines. I would compare the various flight options considering price, duration, layovers (if acceptable), and any other preferences you have mention

Overriding of current TracerProvider is not allowed




# Agent: Mutual Fund Expert
## Final Answer: 
The term "best" mutual funds can vary depending on individual investment goals, risk tolerance, time horizon, and market conditions at the time of investment. As a mutual fund expert, I can guide you towards understanding the different types of mutual funds and factors to consider when selecting a fund to invest in. However, I cannot specify which mutual funds are currently the best without real-time information and a clear understanding of your specific investment profile.

There are several categories of mutual funds that cater to various investment objectives:

1. **Index Funds**: These funds aim to replicate the performance of a specific index, such as the S&P 500. They are known for their low expense ratios and are suitable for passive investors.

2. **Active Funds**: Managed by professional fund managers who actively select securities to outperform the market or a benchmark index. They usually have higher fees than index funds.

3. 

Overriding of current TracerProvider is not allowed




# Agent: General Travel Assistant
## Final Answer: 
To assist you with planning a vacation in Europe, we would need to consider several factors for a comprehensive plan, including destinations, duration, budget, interests, and preferences for accommodations and activities. Here's a general outline to get you started on your European vacation:

1. **Choosing Destinations:**
   - Select countries and cities that interest you.
   - Consider diversity in experiences, from historic cities to natural landscapes.
   - Decide on a travel route that makes sense logistically.

2. **Duration:**
   - Determine how much time you can spend on your vacation.
   - Allocate enough time in each destination to enjoy without feeling rushed.

3. **Budget:**
   - Outline your budget for flights, accommodations, food, activities, and emergencies.
   - Look for travel deals and consider traveling in the shoulder season for lower prices.

4. **Accommodations:**
   - Choose between hotels, hostels, vacation r

Overriding of current TracerProvider is not allowed




# Agent: Stock Market Expert
## Final Answer: 
When considering an investment in tech stocks, it's important to assess several key factors. Firstly, evaluate the current market conditions, as tech stocks can be particularly sensitive to market trends and economic indicators. Look at the performance of the tech sector compared to other sectors, and consider whether tech stocks are currently overvalued or undervalued.

Secondly, tech companies' fundamentals should be analyzed, including their earnings, growth potential, financial health, and competitive positioning. Focus on companies with strong balance sheets, sustainable revenue growth, and innovative products or services that can maintain a competitive edge.

Diversification is also crucial when investing in tech stocks. Instead of concentrating your investment on a single stock or subsector, consider spreading it across various companies and technology subsectors (e.g., software, hardware, semiconductors, cloud computing, artifici